In [1]:
import pandas as pd
import numpy as np
import datetime # for age calculation

In [2]:
# for printing a lot of columns horizontally
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# reads csv into dataframe (uses data/processed folder)
def get_df(filename,sep=","):
    return pd.read_csv("../data/processed/%s" % filename, header=0, sep=sep,low_memory=False)

In [4]:
# reads and cleans a transactions CSV
# returns pandas DataFrame
def get_transactions(filename):
    df = get_df(filename,sep=";")
    #rename opeartions
    df.loc[df["operation"]=="credit in cash","operation"] = "A"
    df.loc[df["operation"]=="collection from another bank","operation"] = "B"
    df.loc[df["operation"]=="withdrawal in cash","operation"] = "C"
    df.loc[df["operation"]=="remittance to another bank","operation"] = "D"
    df.loc[df["operation"]=="credit card withdrawal","operation"] = "E"
    df.loc[df["operation"].isna(),"operation"] = df.loc[df["operation"].isna(),"k_symbol"]
    df.loc[df["operation"]=="interest credited","operation"] = "F"
    # convert "withdrawal in cash" to "withdrawal" in type
    df.loc[df["type"]=="withdrawal in cash","type"] = "withdrawal"
    # withdrawals should be negative values
    df.loc[df["type"]=="withdrawal","amount"] *=-1 
    # remove k_symbol
    df = df.drop(['k_symbol'], axis=1)
    # remove bank and account, as these only record the destination account
    df = df.drop(['bank', 'account'], axis=1)
    # convert date to date value
    df["date"] = pd.to_datetime(df['date'], format='%y%m%d')
    return df

In [5]:
def get_accounts():
    df = get_df("account.csv")
    # simpler values
    df.loc[df["frequency"]=="monthly issuance","frequency"]="monthly"
    df.loc[df["frequency"]=="weekly issuance","frequency"]="weekly"
    df.loc[df["frequency"]=="issuance after transaction","frequency"]="pos-transaction"
    df["creation_date"] = pd.to_datetime(df['creation_date'], format='%Y-%m-%d')
    return df

In [6]:
def get_cards():
    df = get_df("card.csv")
    # convert date to string
    df.issued = df.issued.apply(str)
    # convert date to common format
    df.issued = df.issued.apply(lambda d: "19%2s-%2s-%2s" % (d[0:2], d[2:4], d[4:]))
    df["issued"] = pd.to_datetime(df['issued'], format='%Y-%m-%d')
    return df

In [7]:
def get_dispositions():
    df = get_df("disp.csv")
    # remove disp_id as it is never used, and the primary key is (client_id,account_id)
    df = df.drop(['disp_id'], axis=1)
    df.rename(columns={'type': 'ownership'}, inplace=True)
    return df

In [8]:
def get_clients():
    df = get_df("client.csv")
    df["birthdate"] = pd.to_datetime(df['birthdate'], format='%Y-%m-%d') # string to datetime
    return df

In [9]:
def get_districts():
    df = get_df("district.csv")
    return df

In [10]:
def get_loans(filename):
    df = get_df(filename,sep=";")
    df["date"] = pd.to_datetime(df['date'], format='%d/%m/%Y') # string to datetime
    df.rename(columns={'date': 'loan_date','amount':'loan_amount'}, inplace=True)
    df.status = df.status.fillna('') # for the test data which is unseen
    return df

### Functions for aggregation

In [11]:
def abs_min(x):
    return x.abs().min()
abs_min.__name__ = 'abs_min'

def rangev(x):
    return x.max() - x.min()
rangev.__name__ = 'range'

In [12]:
def count_a(x):
    return sum(x=="A")
def count_b(x):
    return sum(x=="B")
def count_c(x):
    return sum(x=="C")
def count_d(x):
    return sum(x=="D")
def count_e(x):
    return sum(x=="E")
def count_f(x):
    return sum(x=="F")

def count_withdrawal(x):
    return sum(x=="withdrawal")
def count_credit(x):
    return sum(x=="credit")

In [13]:
def mean_a(x):
    return np.mean(x=="A")
def mean_b(x):
    return np.mean(x=="B")
def mean_c(x):
    return np.mean(x=="C")
def mean_d(x):
    return np.mean(x=="D")
def mean_e(x):
    return np.mean(x=="E")
def mean_f(x):
    return np.mean(x=="F")

def mean_withdrawal(x):
    return np.mean(x=="withdrawal")
def mean_credit(x):
    return np.mean(x=="credit")

In [14]:
def std_a(x):
    return np.std(x=="A")
def std_b(x):
    return np.std(x=="B")
def std_c(x):
    return np.std(x=="C")
def std_d(x):
    return np.std(x=="D")
def std_e(x):
    return np.std(x=="E")
def std_f(x):
    return np.std(x=="F")

def std_withdrawal(x):
    return np.std(x=="withdrawal")
def std_credit(x):
    return np.std(x=="credit")

In [15]:
def cov_a(x):
    return np.cov(x=="A")
def cov_b(x):
    return np.cov(x=="B")
def cov_c(x):
    return np.cov(x=="C")
def cov_d(x):
    return np.cov(x=="D")
def cov_e(x):
    return np.cov(x=="E")
def cov_f(x):
    return np.cov(x=="F")

def cov_withdrawal(x):
    return np.cov(x=="withdrawal")
def cov_credit(x):
    return np.cov(x=="credit")

In [16]:
def age_days(x):
    return (x.max()-x.min()).days

# Now we join

## TODO: include card data (how many of which type, how many per disp, ...)
## Há dois distritos diferentes: morada do cliente e localização da sucursal

In [17]:
def merge_loans_with_everything(filename):
    df_l = get_loans(filename)
    df_t = get_transactions("trans.csv") # could be trans_train or trans_test
    df_g = get_districts()
    df_c = get_clients()
    df_d = get_dispositions()
    df_ca = get_cards()
    df_a = get_accounts()

    # Count number of clients per account
    df_ownership_count = df_d.groupby("account_id", as_index=False, group_keys=False).agg({"ownership": ["count"]})
    df_ownership_count.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df_ownership_count.columns]
    df_d = pd.merge(df_d, df_ownership_count, on="account_id")
    # JOIN loan with dispositions
    df_d = df_d[df_d.ownership=="OWNER"] # keep owners only
    df_d = df_d.drop(['ownership'], axis=1) # drop now redundant column
    df = pd.merge(df_l, df_d, on="account_id") # merge disposition with loans
    # JOIN previous with client info
    df = pd.merge(df, df_c, on="client_id") # keep owner and disponent
    # JOIN previous with district info
    df = pd.merge(df, df_g, on="district_id") # keep owner and disponent
    # JOIN previous with district info
    df["creation_date"] = pd.merge(df, df_a, on="account_id")["creation_date"] # keep owner and disponent
    # JOIN previous with transactions
    df = pd.merge(df, df_t, on="account_id")
    df.sort_values(by=["date"], inplace=True)
    return df

In [18]:
df_train = merge_loans_with_everything("loan_train.csv")
df_test = merge_loans_with_everything("loan_test.csv")
df_train.head(10)

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,client_id,ownership_count,gender,birthdate,district_id,name,region,num_inhabitants,num_municipalities_with_inhabitants<499,num_municipalities_with_inhabitants_500-1999,num_municipalities_with_inhabitants_2000-9999,num_municipalities_with_inhabitants>10000,num_cities,ratio_urban_inhabitants,average_salary,unemployment_rate_95,unemployment_rate_96,num_entrepreneurs_per_1000_inhabitants,num_crimes_95,num_crimes_96,creation_date,trans_id,date,type,operation,amount,balance
1988,6077,5270,1993-11-22,79608,24,3317,1,6367,1,M,1970-04-28,44,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,1993-01-13,1548749,1993-01-13,credit,A,800.0,800.0
1030,7284,11265,1993-09-15,52788,12,4399,1,13845,1,M,1973-02-16,15,Cesky Krumlov,south Bohemia,58796,22,16,7,1,5,51.9,9045,3.13,3.60,124,1845,1879,1993-01-14,3393738,1993-01-14,credit,A,1000.0,1000.0
1989,6077,5270,1993-11-22,79608,24,3317,1,6367,1,M,1970-04-28,44,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,1993-01-13,1548750,1993-01-14,credit,B,44749.0,45549.0
1574,7121,10364,1993-11-10,21924,36,609,1,12754,2,M,1960-08-20,55,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894,1993-01-17,3122924,1993-01-17,credit,A,1100.0,1100.0
6977,5754,3834,1994-09-28,23052,12,1921,1,4620,2,F,1940-11-01,54,Brno - mesto,south Moravia,387570,0,0,0,1,1,100.0,9897,1.60,1.96,140,18721,18696,1993-01-19,1121963,1993-01-19,credit,A,700.0,700.0
18154,6895,9307,1994-09-19,41904,12,3492,1,11461,1,M,1974-07-08,70,Karvina,north Moravia,285387,0,2,8,5,7,89.9,10177,6.63,7.75,81,9878,10108,1993-01-24,2809952,1993-01-24,credit,A,900.0,900.0
13441,6202,5891,1994-04-18,65184,12,5432,1,7127,1,M,1957-09-29,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813,1993-01-25,1736607,1993-01-25,credit,A,900.0,900.0
15377,6316,6473,1994-05-26,76908,12,6409,-1,7815,1,F,1946-01-19,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892,1993-01-26,1910780,1993-01-26,credit,A,1100.0,1100.0
15378,6316,6473,1994-05-26,76908,12,6409,-1,7815,1,F,1946-01-19,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892,1993-01-26,1910790,1993-01-26,credit,A,19961.0,21061.0
454,5325,1843,1993-08-03,105804,36,2939,1,2235,1,F,1940-04-20,14,Ceske Budejovice,south Bohemia,177686,69,27,10,1,9,74.8,10045,1.42,1.71,135,6604,6295,1993-01-30,541681,1993-01-30,credit,A,1000.0,1000.0


In [ ]:
# "client_id" is not needed as each client only ever has a single loan
keep_cols = ["loan_id", "account_id", "status", "loan_date", "creation_date", "loan_amount", "duration", "payments", "gender", "birthdate", "ownership_count", "district_id", "num_inhabitants", 'num_municipalities_with_inhabitants<499', 'num_municipalities_with_inhabitants_500-1999', 'num_municipalities_with_inhabitants_2000-9999', 'num_municipalities_with_inhabitants>10000', 'num_cities', 'ratio_urban_inhabitants', 'average_salary', 'unemployment_rate_95', 'unemployment_rate_96', 'num_entrepreneurs_per_1000_inhabitants', 'num_crimes_95', 'num_crimes_96']
def aggregate(df):
    df = df.groupby(keep_cols, as_index=False, group_keys=False).agg({
        "date" : ["max", "min", age_days],
        "operation":["count", count_a, count_b, count_c, count_d, count_e, count_f, 
                     mean_a, mean_b, mean_c, mean_d, mean_e, mean_f,
                     std_a, std_b, std_c, std_d, std_e, std_f,
                     cov_a, cov_b, cov_c, cov_d, cov_e, cov_f],
        "amount": ["mean","min","max","std","last",np.cov,abs_min,rangev],
        "balance":["mean","min","max","std","last",np.cov,abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
    })
    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]
    df["days_last_trans_loan"] = (df["loan_date"] - df["date_max"]).dt.days
    df["last_balance_per_loan"] = df["balance_last"] / df["loan_amount"]
    df["max_balance_per_loan"] = df["balance_max"] / df["loan_amount"]
    df["date_age_months"] = df["date_age_days"]/30
    df["balance_per_month"] = df["balance_range"] / df["date_age_months"]
    df["transactions_per_month"]=df["operation_count"] / df["date_age_months"]
    # calculate client age at loan request
    df['owner_age_at_loan'] = (df['loan_date'] - df['birthdate']).astype('<m8[Y]') # cast to years
    df.owner_age_at_loan = df.owner_age_at_loan.astype(int) # convert to int
    df['account_months_at_loan'] = ((df['loan_date'] - df['creation_date']).dt.days)/30 # cast to months
#     df.account_age_at_loan = df.account_age_at_loan.astype(int) # convert to int
    df = df.drop(['birthdate',"creation_date"], axis=1) #remove birthdate
    
    return df

In [ ]:
df_train2 = aggregate(df_train)
df_test2 = aggregate(df_test)
df_train2.head(20)

In [ ]:
save_to = "../rapid miner/data/data from python/"
df_train2.to_csv("%strain.csv" % save_to, index=False)
df_test2.to_csv("%stest.csv" % save_to, index=False)